# Curso de dbt (Data Build Tool) 

## Introducción a dbt - Transformando el Mundo de los Datos

### Problemas antes de dbt

Antes de dbt, los equipos de datos enfrentaban varios desafíos críticos:

#### 1. **El Problema del "Spaghetti Code"**
- Scripts SQL dispersos sin organización clara
- Transformaciones complejas en archivos monolíticos
- Dependencias implícitas difíciles de rastrear
- Falta de modularidad y reutilización

#### 2. **Ausencia de Mejores Prácticas de Software**
- Testing manual y propenso a errores
- Documentación desactualizada o inexistente
- Colaboración limitada entre equipos

#### 3. **Problemas de Escalabilidad**
- Pipelines frágiles que se rompían frecuentemente
- Dificultad para onboarding de nuevos desarrolladores
- Mantenimiento costoso y tiempo de desarrollo lento

### ¿Qué es dbt? 

**dbt (Data Build Tool) es una herramienta de línea de comandos que permite a los equipos de datos transformar datos en sus warehouses aplicando prácticas de ingeniería de software**.

#### Definición Técnica
dbt es un **framework de transformación de datos** que:
- Compila modelos en SQL puro
- Ejecuta transformaciones directamente en el data warehouse
- Gestiona dependencias automáticamente
- Aplica testing y documentación de forma nativa

#### Filosofía Core de dbt
```
"dbt does the T in ELT"
(Extract, Load, Transform)
```

En lugar del ETL tradicional, dbt se centra en el modelo **ELT**:
- **Extract & Load**: Ingestar los datos de origen con transformaciones minimas.
- **Transform**: dbt se encarga de las transformaciones.

### Los Modelos: El Corazón de dbt

#### ¿Qué es un Modelo?
Un **modelo** en dbt es:
- Un archivo `.sql` que contiene una consulta SELECT donde se combina SQL y Jinja
- Representa una transformación de datos
- Se ejecuta y materializa en el warehouse

#### Ejemplo Conceptual
```sql
select
    customer_id,
    first_name,
    last_name,
    email
from {{ source('raw_data', 'customers') }}
where active = true
```

### Funciones Especiales que Revolucionan el SQL

#### 1. **ref() - Referencias entre Modelos**
```sql
-- Crea dependencias automáticas
select * from {{ ref('staging__customers') }}
```

#### 2. **source() - Referencias a Datos Raw**
```sql
-- Referencia a tablas originales
select * from {{ source('ecommerce', 'raw_orders') }}
```


### Componentes Arquitectónicos de dbt

#### 1. **El Compilador (Compiler)**
- **Función**: Convierte los modelos en SQL puro
- **Proceso**:
  ```
  Módelo dbt → Compilador → SQL ejecutable
  ```

#### 2. **El Runner (Ejecutor)**
- **Función**: Ejecuta el SQL compilado en el data warehouse
- **Características**:
  - Gestión automática de dependencias gracias al uso de DAGs.
  - Ejecución paralela cuando es posible
  - Manejo de errores y rollbacks
  - Logging detallado de operaciones



## Estructura del Proyecto jaffle_shop

Nuestro proyecto tiene la siguiente estructura:

```
jaffle_shop/
├── dbt_project.yml          # Configuración del proyecto
├── profiles.yml             # Configuración de conexión
├── models/
│   ├── staging/             # Modelos de preparación (views)
│   └── marts/               # Modelos finales (tables)
├── macros/                  # Funciones reutilizables
├── tests/                   # Tests personalizados
└── seeds/                   # Datos raw
```


## Comandos Básicos de dbt

### Comandos Esenciales

```bash
# Verificar configuración
dbt debug

# Cargar ficheros seed
dbt seed

# Ejecutar todos los modelos
dbt run

# Ejecutar un modelo específico
dbt run --models staging__order_items

# Ejecutar modelos con dependencias
dbt run --models marts__customers+

# Compilar sin ejecutar
dbt compile

# Generar fichero yml
dbt run-operation generate_model_yaml --args '{"model_names": ["staging__customers"]}'

# Ver la documentación
dbt docs generate
dbt docs serve
```

Con `dbt seed` podemos cargar datos de ejemplo en nuestro data warehouse desde archivos CSV ubicados en la carpeta `seeds/`. Estos datos serán datos en raw, los datos de partida, nuestras sources.

Ahora imaginemos que estamos interesados en rresponder las siguientes preguntas de negocio:

1. ¿Cuántos clientes activos tenemos (con compras en el último mes)?
2. ¿Cuál es el total de ventas por producto?
3. ¿Cuáles son los 10 productos más vendidos en el último trimestre?

Vamos a resolverlos utilizando las tablas que tenemos ahora mismo (las tablas de raw)

Podremos resolverlo con querys como las siguientes: 

```sql
-- 1. Clientes activos
select 
  count(distinct customer) as number_of_active_customers 
from 
  jaffle_shop.raw.orders
where 
  ORDERED_AT >= '2016-08-01'
```
```sql
-- 2. Total de ventas por producto
select 
    sku, 
    sum(price) as total_ventas
from 
    jaffle_shop.raw.items
group by 
    sku
order by 
    total_ventas desc
```
```sql
-- 3. Top 10 productos más vendidos desde 2016-07-01
select 
    sku, 
    sum(units) as unidades_vendidas
from
    jaffle_shop.raw.items
inner join 
  jaffle_shop.raw.orders
on orders.id = items.order_id
where 
    ORDERED_AT >= '2016-07-01' -- Esta comparación puede ser peligrosa
group by 
    sku
order by 
    unidades_vendidas desc
limit 10
```

## Capa STAGING

Dado que los datos en las tablas raw no están limpios ni estructurados para consultar, el primer paso es crear una capa de staging. Esta capa se encargará de preparar y limpiar los datos para que estén listos para su consulta. La capa staging estará compuesta por modelos dbt que transformen las tablas raw en vistas limpias y estructuradas. Es aqui donde tendremos el primer acercamiento con los modelos de dbt y donde podremos ver la filosofia de dbt en acción.


### El Paradigma dbt: Transform First, Questions Later

#### Enfoque Tradicional
```
Pregunta de Negocio → Análisis Ad-hoc → SQL específico → Resultado
```

#### Enfoque dbt
```
Datos en bruto → Modelos Base → Modelos Intermedios → Modelos Marts → Múltiples Análisis
```

**Ventaja**: Una vez construidos los modelos base, responder nuevas preguntas es mucho más rápido.

Ahora ya podemos trabajar con las tablas limpias desde el inicio, sin tener que lidiar con los datos sucios de las tablas raw. Pero aún así, para responder ciertas preguntas, tenemos que hacer transformaciones adicionales. Por ejemplo, para calcular el total de ventas por producto y mes, necesitamos unir las tablas de orders e items y realizar agregaciones. Dado que queremos aislar toda la complejidad posible, crearemos una nueva capa intermedia llamada "intermediate" donde realizaremos estas transformaciones adicionales antes de llegar a la capa final.

## Capa Intermediate

La capa intermediate sirve como puente entre la capa de staging y la capa de marts. En esta capa, realizamos transformaciones adicionales y agregaciones que preparan los datos para su análisis final. Los modelos en esta capa suelen ser más complejos y pueden incluir cálculos específicos del negocio. Añadiremos mayor complejidad, por lo que se distingue de la capa de staging, pero no serán tablas completas para un consumo final, por lo que no estarán en la capa de marts.

En este caso crearemos las tablas intermedias:

- intermediate__orders_lineitems: Combina las tablas de orders e items, calculando cantidades originales y finales, precios unitarios y subtotales, y si hubo devoluciones.
- intermediate__orders: Agrega los datos combinados por fecha, ubicación y cliente, contando el número de productos y sumando cantidades y precios.

Ambas presentan una complejidad considerable (una a nivel de producto y otra a nivel de order), pero aún no están listas para el consumo final, por lo que las ubicamos en la capa intermediate.


## Capa Marts

La capa marts es la capa final donde los datos están completamente preparados para el análisis y la generación de informes. Los modelos en esta capa suelen ser tablas que contienen métricas clave y dimensiones que los analistas pueden utilizar directamente para responder preguntas de negocio. En esta capa, los modelos son optimizados para el rendimiento y la facilidad de uso. Dado que esta capa está destinada a resolver preguntas, debemos pensar en la estructura de las tablas para que sean intuitivas y fáciles de consultar. 

Por ejemplo, imaginemos que queremos reponder preguntas como las siguientes:

1. ¿Cuántas compras ha hecho cada cliente en el último año?
2. ¿Cuál es el total de ventas por producto de los sacados recientemente?
3. ¿Cuál es el precio de los 10 productos más vendidos en el último año?
4. ¿Qué porcentaje de la compra suponen los productos de comida para cada cliente?

Ahora que ya hemos limpiado y enriquecido nuestras tablas, toca crear la última capa: la capa marts. En esta capa, crearemos modelos que respondan directamente a las preguntas de negocio planteadas anteriormente.

## Configuración de los Modelos en dbt

En dbt, podemos configurar cómo se comportan y materializan nuestros modelos usando el **bloque de configuración (config)**. Esta configuración determina aspectos cruciales como el tipo de materialización, el esquema de destino, las etiquetas, y mucho más.

### Formas de Configurar un Modelo

#### 1. Configuración en el archivo SQL (Recomendado para configuraciones específicas)
```sql
{{ config(
    materialized='table',
    schema='staging',
    tags=['daily', 'critical']
) }}

select
    customer_id,
    first_name,
    last_name
from {{ source('raw_data', 'customers') }}
```

#### 2. Configuración en dbt_project.yml (Para configuraciones globales)
```yaml
models:
  jaffle_shop:
    staging:
      +materialized: view
      +schema: staging
    marts:
      +materialized: table
      +schema: analytics
```

#### 3. Configuración en archivos .yml de propiedades (menos usado)
```yaml
models:
  - name: staging__customers
    config:
      materialized: view
      tags: ['pii', 'hourly']
```

### Parámetros de Configuración Más Comunes

#### 1. **materialized** - Tipo de Materialización
Define cómo se crea el modelo en el warehouse:

```sql
-- Vista (recalcula en cada consulta)
{{ config(materialized='view') }}

-- Tabla física (se reconstruye completamente)
{{ config(materialized='table') }}

-- Incremental (solo añade/actualiza nuevos registros)
{{ config(materialized='incremental') }}

-- Ephemeral (solo existe en compilación, no se crea en warehouse)
-- Equivalente a un alias
{{ config(materialized='ephemeral') }}
```

#### 2. **schema** - Esquema de Destino
```sql
{{ config(schema='staging') }}

-- Se creará en: <target_schema>_staging
-- Ejemplo: jaffle_shop_staging
```

#### 3. **alias** - Nombre Personalizado de la Tabla
```sql
{{ config(alias='customer_data') }}

-- En lugar de "staging__customers"
-- Se creará como "customer_data"
```

#### 4. **tags** - Etiquetas para Organización
```sql
{{ config(tags=['daily', 'core', 'pii']) }}

-- Permite ejecutar grupos de modelos:
-- dbt run --models tag:daily
-- dbt run --models tag:pii
```

#### 5. **enabled** - Habilitar/Deshabilitar Modelo
```sql
{{ config(enabled=false) }}

-- El modelo no se ejecutará ni compilará
-- Útil para deprecar modelos temporalmente
```

#### 6. **unique_key** - Clave para Modelos Incrementales
```sql
{{
    config(
        materialized='incremental',
        unique_key='order_id'
    )
}}

select * from {{ ref('staging__orders') }}

{% if is_incremental() %}
where order_date > (select max(order_date) from {{ this }})
{% endif %}
```

#### 7. **pre_hook** y **post_hook** - Hooks SQL
```sql
{{
    config(
        pre_hook="grant usage on schema {{ schema }} to role analyst",
        post_hook="grant select on {{ this }} to role analyst"
    )
}}

-- Se ejecutan antes/después de materializar el modelo
```

### Precedencia de Configuración

La configuración se aplica en el siguiente orden (de menor a mayor prioridad):

1. **dbt_project.yml** (configuración global)
2. **Archivos .yml de propiedades** (configuración por modelo)
3. **Bloque config en archivo .sql** (configuración específica)
4. **Línea de comandos** (dbt run --vars)

```sql
-- Esto sobrescribe configuraciones previas
{{ config(materialized='table') }}

-- Incluso si dbt_project.yml dice:
-- +materialized: view
```

### Mejores Prácticas

1. **Usa config en SQL para configuraciones específicas del modelo**
   - Materializaciones especiales
   - unique_key para incrementales

2. **Usa dbt_project.yml para configuraciones por carpeta**
   - Todos los staging como views
   - Todos los marts como tables

3. **Documenta decisiones de configuración**
   ```sql
   {{ config(
       materialized='incremental',
       -- Incremental porque esta tabla tiene >100M filas
       -- y se actualiza cada hora
       unique_key='log_id'
   ) }}
   ```

4. **Usa tags para organización y ejecución selectiva**
   ```bash
   dbt run --models tag:daily     # Solo modelos diarios
   dbt run --models tag:pii       # Solo datos sensibles
   dbt run --models tag:critical  # Solo tablas críticas
   ```


## Documentación en dbt

### ¿Por qué es Importante la Documentación?

Antes de dbt, la documentación de datos era:
- ❌ Manual y propensa a quedar desactualizada
- ❌ Dispersa en wikis, Google Docs o Confluence
- ❌ Difícil de mantener sincronizada con el código
- ❌ Poco accesible para analistas de negocio

**dbt revoluciona la documentación al:**
- ✅ Generarla automáticamente desde el código
- ✅ Mantenerla junto al código (fuente única de la verdad)
- ✅ Crear un sitio web interactivo con linaje de datos
- ✅ Facilitar la colaboración entre equipos técnicos y de negocio

---

### Generación Automática de Archivos YAML

Antes de documentar manualmente, dbt ofrece herramientas para **generar automáticamente la estructura base de los archivos YAML**. Esto ahorra tiempo y asegura que no olvidemos ninguna columna.

#### El Paquete `codegen`

**codegen** es un paquete de dbt que proporciona macros para generar código automáticamente, incluyendo archivos YAML de documentación.

##### Instalación

1. **Agregar el paquete en `packages.yml`:**
```yaml
# packages.yml (en la raíz del proyecto)
packages:
  - package: dbt-labs/codegen
    version: 0.12.1
```

2. **Instalar las dependencias:**
```bash
dbt deps
```

Esto descargará el paquete codegen en la carpeta `dbt_packages/`.

#### Comando: `generate_model_yaml`

Este comando genera automáticamente la estructura YAML para uno o varios modelos, incluyendo:
- Nombre del modelo
- Lista completa de columnas
- Tipos de datos

##### Sintaxis Básica

```bash
dbt run-operation generate_model_yaml --args '{"model_names": ["nombre_modelo"]}'
```

##### Ejemplos Prácticos

**Ejemplo 1: Generar YAML para un solo modelo**

```bash
cd jaffle_shop
dbt run-operation generate_model_yaml --args '{"model_names": ["staging__customers"]}'
```

**Ejemplo 2: Generar YAML para múltiples modelos**

```bash
dbt run-operation generate_model_yaml --args '{"model_names": ["staging__customers", "staging__orders", "staging__products"]}'
```

#### Flujo de Trabajo Recomendado

1. **Crear el modelo SQL**
   ```sql
   -- models/staging/staging__customers.sql
   select
       id as customer_id,
       first_name,
       last_name,
       email
   from {{ source('raw', 'customers') }}
   ```

2. **Ejecutar el modelo para crear la tabla**
   ```bash
   dbt run --models staging__customers
   ```

3. **Generar el YAML automáticamente**
   ```bash
   dbt run-operation generate_model_yaml --args '{"model_names": ["staging__customers"]}'
   ```

4. **Copiar la salida a un archivo YAML**
   ```bash
   # Crear o editar el archivo
   # models/staging/staging__customers.yml
   ```
   
   Pegar la salida generada y guardar.

5. **Completar las descripciones manualmente**
   ```yaml
   version: 2

   models:
     - name: staging__customers
       description: "Datos limpios de clientes desde la tabla raw"
       columns:
         - name: customer_id
           description: "Identificador único del cliente (PK)"
           tests:
             - unique
             - not_null
         
         - name: first_name
           description: "Nombre del cliente"
         
         - name: last_name
           description: "Apellido del cliente"
         
         - name: email
           description: "Email de contacto del cliente"
           tests:
             - unique
   ```

#### Ventajas de la Generación Automática

✅ **Ahorro de tiempo**: No escribir manualmente cada columna

✅ **Cero errores de tipeo**: Los nombres se extraen directamente del warehouse

✅ **Completitud**: No olvidarás documentar ninguna columna

✅ **Punto de partida**: Estructura base lista para agregar descripciones

✅ **Escalabilidad**: Puedes generar YAML para decenas de modelos rápidamente

#### Limitaciones

⚠️ **No genera descripciones**: Las descripciones quedan vacías, debes completarlas manualmente

⚠️ **No añade tests**: Debes agregar los tests según tus necesidades

⚠️ **Sobrescribe**: Ten cuidado de no sobrescribir YAML existente con documentación

---

### El Comando dbt docs

Una vez que tienes tus archivos YAML (generados automáticamente o escritos manualmente), dbt puede crear un sitio web de documentación interactivo.

dbt incluye dos comandos principales para documentación:

#### 1. `dbt docs generate`
```bash
dbt docs generate
```

**¿Qué hace?**
- Extrae información de los modelos, tests, fuentes y macros
- Lee las descripciones de los archivos `.yml`
- Genera metadatos sobre columnas, tipos de datos y dependencias
- Crea archivos JSON con toda la información (`manifest.json` y `catalog.json`)

**Archivos generados:**
```
target/
├── manifest.json      # Estructura del proyecto, dependencias, configuraciones
└── catalog.json       # Metadatos del warehouse (tipos de columnas, estadísticas)
```

#### 2. `dbt docs serve`
```bash
dbt docs serve
```

**¿Qué hace?**
- Inicia un servidor web local (generalmente en `http://localhost:8080`)
- Presenta un sitio interactivo con toda la documentación
- Permite navegar por modelos, dependencias y linaje de datos
- Incluye búsqueda y filtrado avanzado

**Para detenerlo:** Presiona `Ctrl+C` en la terminal

---

### Cómo Documentar tus Modelos Manualmente

Después de generar la estructura base con codegen, es importante completar y enriquecer la documentación.

#### 1. Documentación Básica en Archivos YAML

```yaml
# models/staging/schema.yml
version: 2

models:
  - name: staging__customers
    description: |
      Datos de clientes limpios y estandarizados desde la tabla raw.
      Cada registro representa un cliente único.
      
      **Transformaciones aplicadas:**
      - Renombrado de columnas a snake_case
      - Conversión de tipos de datos
      - Eliminación de registros duplicados
    
    columns:
      - name: customer_id
        description: "Identificador único del cliente (PK)"
        tests:
          - unique
          - not_null
      
      - name: first_name
        description: "Nombre del cliente"
      
      - name: last_name
        description: "Apellido del cliente"
      
      - name: email
        description: "Email de contacto del cliente"
        tests:
          - unique
          - not_null
```

#### 2. Documentación de Fuentes (Sources)

```yaml
# models/staging/sources.yml
version: 2

sources:
  - name: raw
    description: "Datos crudos importados del sistema transaccional"
    database: jaffle_shop
    schema: raw
    
    tables:
      - name: customers
        description: |
          Tabla de clientes del sistema de e-commerce.
          Actualizada cada noche a las 2:00 AM UTC.
        
        columns:
          - name: id
            description: "ID único del cliente"
            tests:
              - unique
              - not_null
          
          - name: first_name
            description: "Nombre del cliente"
```

#### 3. Documentación Avanzada con Metadatos

```yaml
models:
  - name: marts__customers
    description: "Tabla analítica de métricas agregadas por cliente"
    
    meta:
      owner: "Data Analytics Team"
      update_frequency: "Daily at 3:00 AM"
      maturity: "production"
      contains_pii: true
    
    columns:
      - name: customer_id
        description: "Identificador único del cliente"
        meta:
          sensitive: false
      
      - name: lifetime_spend
        description: |
          Gasto total del cliente desde su primer pedido.
          **Fórmula:** Suma de subtotal + tax + shipping de todos los pedidos
        meta:
          unit: "USD"
          calculation: "sum(order_total)"
```

### Características del Sitio de Documentación

#### 1. **Vista General del Proyecto**
- Lista de todos los modelos organizados por carpeta
- Estadísticas del proyecto (número de modelos, tests, fuentes)
- Búsqueda global por nombre o descripción

#### 2. **Vista de Modelo Individual**
Cada modelo muestra:
- Descripción detallada
- Lista de columnas con tipos y descripciones
- Tests configurados
- Código SQL compilado
- Linaje de datos (dependencies upstream y downstream)

#### 3. **Gráfico de Linaje (DAG)**
- Visualización interactiva de dependencias
- Navegación por el grafo de transformaciones
- Identificación de modelos upstream (fuentes) y downstream (consumidores)

**Ejemplo de navegación:**
```
raw_customers → staging__customers → intermediate__customer_orders → marts__customers
```

#### 4. **Documentación de Macros**
- Descripción de macros personalizados
- Parámetros de entrada y salida
- Ejemplos de uso

#### 5. **Información del Warehouse**
- Tipos de datos de cada columna (extraídos del warehouse)
- Número de filas (si está disponible)
- Última actualización

### Flujo de Trabajo Completo Recomendado

```bash
# 1. Desarrollar modelo SQL
# models/marts/marts__customers.sql

# 2. Ejecutar modelo
dbt run --models marts__customers

# 3. Generar YAML base automáticamente
dbt run-operation generate_model_yaml --args '{"model_names": ["marts__customers"]}'

# 4. Copiar salida a schema.yml y añadir descripciones y tests
# models/marts/schema.yml

# 5. Generar documentación
dbt docs generate

# 6. Visualizar documentación
dbt docs serve

# 7. Revisar en navegador (http://localhost:8080)
# - Ver descripción del modelo
# - Revisar linaje de datos
# - Validar que columnas están documentadas
```

### Mejores Prácticas de Documentación

#### 1. **Documenta el "Por Qué", No el "Qué"**
```yaml
# ❌ Malo - describe lo obvio
- name: customer_count
  description: "Número de clientes"

# ✅ Bueno - explica el propósito y lógica
- name: customer_count
  description: |
    Número de clientes únicos activos en los últimos 90 días.
    Un cliente se considera activo si ha realizado al menos un pedido.
```

#### 2. **Documenta Decisiones de Negocio**
```yaml
models:
  - name: marts__revenue
    description: |
      Métricas de ingresos para reportes ejecutivos.
      
      **Nota importante:** Los ingresos excluyen pedidos cancelados
      y devoluciones, siguiendo la definición de CFO del Q2 2024.
      
      Para ingresos brutos (incluyendo devoluciones), usar
      marts__revenue_gross en su lugar.
```

#### 3. **Usa Markdown para Formato**
```yaml
columns:
  - name: customer_tier
    description: |
      Segmentación del cliente basada en lifetime value:
      
      - **VIP**: LTV > $10,000
      - **Premium**: LTV entre $5,000 - $10,000
      - **Standard**: LTV entre $1,000 - $5,000
      - **New**: LTV < $1,000
```

#### 4. **Documenta Transformaciones Complejas**
```yaml
- name: churn_probability
  description: |
    Probabilidad de abandono del cliente en los próximos 30 días.
    
    **Modelo:** Regresión logística entrenada en datos históricos
    **Variables:** días desde último pedido, frecuencia de compra, ticket promedio
    **Última actualización del modelo:** 2024-01-15
```

#### 5. **Mantén Documentación Cercana al Código**
```
models/
├── marts/
│   ├── marts__customers.sql
│   └── marts__customers.yml          # Documentación en el mismo directorio
```

### Beneficios de una Buena Documentación

1. **Onboarding más rápido**: Nuevos analistas entienden el modelo de datos rápidamente
2. **Colaboración mejorada**: Equipos técnicos y de negocio hablan el mismo idioma
3. **Debugging más fácil**: El linaje de datos ayuda a rastrear problemas
4. **Confianza en los datos**: Transparencia sobre qué representan las métricas
5. **Self-service analytics**: Analistas pueden explorar datos sin ayuda constante


Ahora que sabes cómo generar y documentar tus modelos, veamos cómo validar que funcionan correctamente con **testing**.

## Testing en dbt

### Tipos de Tests

#### 1. Tests Genéricos
- `unique`: Valores únicos
- `not_null`: Sin valores nulos
- `accepted_values`: Valores permitidos
- `relationships`: Integridad referencial

#### 2. Tests Personalizados
```sql
-- tests/assert_no_negative_customer_metrics.sql
select customer_id
from {{ ref('marts__customers') }}
where lifetime_spend < 0
   or count_lifetime_orders < 0
```

### Ejemplo de Configuración de Tests
```yaml
# tests/properties.yml
models:
  - name: marts__customers
    columns:
      - name: customer_id
        tests:
          - unique
          - not_null
      - name: customer_type
        tests:
          - accepted_values:
              values: ['new', 'returning']
```

### Ejecutar Tests
```bash
# Todos los tests
dbt test

# Tests de un modelo específico
dbt test --models marts__customers
```